Porcentaje de personas que se dedican a actividades de cuidados y no reciben remuneración

In [ ]:
##
rm(list=ls())
#Paquetes y liberías
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr, readr, stringr, dplyr, utils, survey)
# URL del archivo ZIP
url <- "https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_poblacion_csv.zip"

In [2]:
##Creación de directorio temporal
td<- tempdir()
# Descarga del archivo temporal
tf = tempfile(tmpdir=td, fileext=".zip")
download.file(url, tf)
unzip(tf, files="poblacion.csv", exdir=td, 
      overwrite=TRUE)
fpath=file.path(td,"poblacion.csv")
unlink(td)

In [ ]:
pob <- read_csv(fpath, show_col_types = FALSE)
names(pob)
datos2 <- pob [ ,c("folioviv", "foliohog", "hor_4", "est_dis", "entidad", "edad", "upm", "factor")]
#variable dicotómica
datos <- datos2 %>% 
  filter(entidad == "09")|>
  mutate(pob_interes = case_when(hor_4 != 0 | !is.na(hor_4) ~ 1, TRUE ~ 0))


In [6]:
## Diseño muestral
dm <- datos %>%
  as_survey_design(
    ids = upm,
    strata = est_dis,
    weights = factor,
    nest = TRUE)

In [14]:
pob_cuidados<- dm|>
  group_by(pob_interes)|>
  summarise(pob_cuid = survey_total(vartype = "cv"),
            porc_cuid = survey_prop(vartype = "cv")*100)|>
  #mutate(prc_p_cuid =pob_cuid*100/sum(pob_cuid))|> PUEDES OBTENER EL PORCENTAJE DE FORMA TRADICIONAL
  filter(pob_interes == 1)

In [16]:
head(pob_cuidados)


pob_interes,pob_cuid,pob_cuid_cv,porc_cuid,porc_cuid_cv
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1569286,0.05277464,16.78938,4.772984
